# Analysis of mitigation potential of five strategies:

 1. Decarbonization of on-site electricity consumption (switch to wind power)
 2. Decarbonization of process heat (electrification of heat)
 3. Decarbonization of electricity in chemical indsutry (switch to wind power)
 4. Decarbonization of process heat in chemical industry (electrification of heat)
 5. Decarbonization of global power (switch to wind)

### Basic set-up

In [1]:
%run _imports.ipynb

In [2]:
BW_PROJECT = 'iri_work' # insert your project name here
bw.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent 3.9.1 cutoff' # name of ecoinvent database in your project
METALS_DB = "EV battery metals"

INVENTORIES = {
    "Lithium, brine":        ("lithium carbonate production, from Salar de Atacama", "lithium carbonate, battery grade", "CL"),
    "Lithium, spodumene":    ("lithium carbonate production, from Greenbushes mine", "lithium carbonate, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper mining, economic allocation", "cobalt sulfate", "CN"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
    "Graphite, synthetic":   ("synthetic graphite production, graphitization", "synthetic graphite, 99.9% pure", "CN")
}

IMPACT_METHODS = {
    "Climate change": ('IPCC 2021', 'climate change: including SLCFs', 'global warming potential (GWP100)')
 }

technosphere = lambda x: x["type"] == "technosphere"

breakdown_lists = supporting_functions.get_breakdown_lists()

In [3]:
dbs_strategies = {
    "BAU": METALS_DB,
    "ONSITE_ELEC": METALS_DB + ", wind power on-site",
    "PROCESS_HEAT": METALS_DB + ", + electrification process heat",
    "CHEMICALS_ELEC": METALS_DB + ", + wind power in chemical industry",
    "CHEMICALS_HEAT": METALS_DB + ", + electrification heat in chemical industry",
    "GLOBAL_ELEC": METALS_DB + ", + wind power in all activities",
}

# Datasets used to model decarbonization strategies

# On-site electricity source shifts to wind onshore
wind_filter = {
    'name': "electricity production, wind, 1-3MW turbine, onshore",
    'product': "electricity, high voltage",
    'unit': "kilowatt hour"
    }

# Process heat is electrified
heat_filter = {
    'name': "heat production, at heat pump 30kW, allocation exergy",
    'product': "heat, central or small-scale, other than natural gas",
    'unit': "megajoule"
    }

In [4]:
# Import ecoinvent database into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(ei_db)
except NameError:
  ei_db = wurst.extract_brightway2_databases(EI_DB)

# Import metals inventories into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(metals_db)
except NameError:
  metals_db = wurst.extract_brightway2_databases(METALS_DB)

Getting activity data


100%|██████████| 21238/21238 [00:00<00:00, 138006.05it/s]


Adding exchange data to activities


100%|██████████| 674593/674593 [00:51<00:00, 13107.81it/s]


Filling out exchange data


...

100%|██████████| 25/25 [00:00<00:00, 12499.42it/s]


Adding exchange data to activities


100%|██████████| 598/598 [00:00<00:00, 17845.69it/s]


Filling out exchange data


100%|██████████| 25/25 [00:00<00:00, 106.00it/s]


In [5]:
# Copy of databases for modification:
ei_db_modif = copy.deepcopy(ei_db)
metals_db_modif = copy.deepcopy(metals_db)

### 1. Decarbonization of on-site electricity consumption

In [79]:
for ds in metals_db_modif:
    if ds["location"] == "GLO":
        pass
    else:
        wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, ds["location"], ei_db)

        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in ['electricity, low voltage',
                                            "electricity, medium voltage",
                                            "electricity, high voltage"]:
                exc.update({
                    'name': wind_ds_loc['name'],
                    'product': wind_ds_loc['reference product'],
                    'location': wind_ds_loc['location'],
                    'input': (wind_ds_loc['database'], wind_ds_loc['code'])})

In [80]:
if dbs_strategies["ONSITE_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["ONSITE_ELEC"]]
wurst.write_brightway2_database(metals_db_modif, dbs_strategies["ONSITE_ELEC"])

Writing activities to SQLite3 database:


25 datasets
598 exchanges
0 unlinked exchanges
  


0% [#########################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 13:11:17
  Finished: 12/22/2023 13:11:17
  Total time elapsed: 00:00:00
  CPU %: 99.20
  Memory %: 1.77
Created database: EV battery metals, wind power on-site


### 2. Electrification of process heat

In [81]:
# Create regionalized heat pump datasets:
list_countries = list(set([ds["location"] for ds in metals_db_modif if ds["location"] != "GLO"]))

# Get heat pump dataset
heat_pump_ds = supporting_functions.get_dataset_for_location(heat_filter, "RoW", ei_db)

for c in list_countries:
    # Get wind power
    wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, c, ei_db)

    # Create heat pump dataset for location and substitute electricity source by wind
    heat_pump_loc = supporting_functions.replicate_activity_to_location(heat_pump_ds, c, ei_db, METALS_DB)

    for exc in filter(technosphere, heat_pump_loc["exchanges"]):
        if exc['product'] in ['electricity, low voltage',
                                "electricity, medium voltage",
                                "electricity, high voltage"]:
            exc.update({
                    'name': wind_ds_loc['name'],
                    'product': wind_ds_loc['reference product'],
                    'location': wind_ds_loc['location'],
                    'input': (wind_ds_loc['database'], wind_ds_loc['code'])})

    metals_db_modif.append(heat_pump_loc)

In [82]:
for ds in metals_db_modif:
    if ds["location"] == "GLO":
        pass
    else:
        heat_pump_loc = supporting_functions.get_dataset_for_location(heat_filter, ds["location"], metals_db_modif)

        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in breakdown_lists["heating products"]:
                exc.update({
                    'name': heat_pump_loc['name'],
                    'product': heat_pump_loc['reference product'],
                    'location': heat_pump_loc['location'],
                    'input': (heat_pump_loc['database'], heat_pump_loc['code'])})

In [83]:
if dbs_strategies["PROCESS_HEAT"] in bw.databases:
    del bw.databases[dbs_strategies["PROCESS_HEAT"]]
wurst.write_brightway2_database(metals_db_modif, dbs_strategies["PROCESS_HEAT"])

Writing activities to SQLite3 database:


29 datasets
614 exchanges
0 unlinked exchanges
  


0% [#############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 13:11:36
  Finished: 12/22/2023 13:11:36
  Total time elapsed: 00:00:00
  CPU %: 99.70
  Memory %: 1.83
Created database: EV battery metals, + electrification process heat


### 3. Decarbonization of electricity in chemical industry

In [84]:
EI_CHEMICALS_WIND = "ecoinvent 3.9.1 cutoff, chemicals production powered by wind"

In [11]:
# Get list of product categories corresponding to chemicals
product_categories = []
used_reagents_categories = []

for ds in ei_db_modif:
    product_category = [i[1] for i in ds["classifications"] if i[0] == 'ISIC rev.4 ecoinvent']
    if len(product_category) > 0:
        product_categories.append(product_category[0])

        if ds["reference product"] in breakdown_lists["reagent products"]:
            used_reagents_categories.append(product_category[0])
        
product_categories = list(set(product_categories))
used_reagents_categories = list(set(used_reagents_categories))

In [12]:
used_reagents_categories

['2420:Manufacture of basic precious and other non-ferrous metals',
 '0729:Mining of other non-ferrous metal ores',
 '0891:Mining of chemical and fertilizer minerals',
 '2394:Manufacture of cement, lime and plaster',
 '2013:Manufacture of plastics and synthetic rubber in primary forms',
 '2030:Manufacture of man-made fibres',
 '2011:Manufacture of basic chemicals',
 '0810:Quarrying of stone, sand and clay',
 '1920:Manufacture of refined petroleum products',
 '2012:Manufacture of fertilizers and nitrogen compounds',
 '3830:Materials recovery',
 '0620:Extraction of natural gas',
 '1040:Manufacture of vegetable and animal oils and fats']

In [13]:
chemicals_ISIC_categories = [
    '1920:Manufacture of refined petroleum products',
    '1040:Manufacture of vegetable and animal oils and fats',
    '2394:Manufacture of cement, lime and plaster',
    '0891:Mining of chemical and fertilizer minerals',
    '2011:Manufacture of basic chemicals',
    '0810:Quarrying of stone, sand and clay',
    '2012:Manufacture of fertilizers and nitrogen compounds',
    '2030:Manufacture of man-made fibres',
    '2013:Manufacture of plastics and synthetic rubber in primary forms',
    '2100:Manufacture of pharmaceuticals, medicinal chemical and botanical products',
    '2029:Manufacture of other chemical products n.e.c.',
    '2220:Manufacture of plastics products',
    '201:Manufacture of basic chemicals, fertilizers and nitrogen compounds, plastics',
    '20:Manufacture of chemicals and chemical products',
    '2023:Manufacture of soap and detergents, cleaning and polishing preparations, pe'
]

In [14]:
for ds in ei_db_modif:
    product_category = [i[1] for i in ds["classifications"] if i[0] == 'ISIC rev.4 ecoinvent']

    if len(product_category) > 0:
        if product_category[0] in chemicals_ISIC_categories:

            # Get wind power
            wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, ds["location"], ei_db_modif)

            for exc in filter(technosphere, ds["exchanges"]):
                if exc['product'] in ['electricity, low voltage',
                                    "electricity, medium voltage",
                                    "electricity, high voltage"]:
                    exc.update({
                        'name': wind_ds_loc['name'],
                        'product': wind_ds_loc['reference product'],
                        'location': wind_ds_loc['location'],
                        'input': (wind_ds_loc['database'], wind_ds_loc['code'])}
                    )

In [25]:
# Write the modified ecoinvent database to memory
if DB_CHEMICALS_WIND in bw.databases:
    del bw.databases[DB_CHEMICALS_WIND]
wurst.write_brightway2_database(ei_db_modif, EI_CHEMICALS_WIND)

Vacuuming database 
21238 datasets
674593 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 12:07:11
  Finished: 12/22/2023 12:07:56
  Total time elapsed: 00:00:44
  CPU %: 81.00
  Memory %: 26.45
Created database: ecoinvent 3.9.1 cutoff, chemicals production powered by wind


In [85]:
# Link metal inventories to the modified ecoinvent version
for ds in metals_db_modif:
    for exc in filter(technosphere, ds["exchanges"]):
        if "ecoinvent" in exc["database"]:
            exc_ds = [
                d for d in bw.Database(EI_CHEMICALS_WIND) if 
                d["name"] == exc["name"] and
                d["reference product"] == exc["product"] and
                d["location"] == exc["location"] and
                d["unit"] == exc["unit"]
                ][0]
                
            exc.update(
                {
                    "input": (exc_ds["database"], exc_ds["code"])
                }
            )

In [86]:
if dbs_strategies["CHEMICALS_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["CHEMICALS_ELEC"]]
wurst.write_brightway2_database(metals_db_modif, dbs_strategies["CHEMICALS_ELEC"])

29 datasets
614 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [#############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 13:20:25
  Finished: 12/22/2023 13:20:25
  Total time elapsed: 00:00:00
  CPU %: 71.70
  Memory %: 20.72
Created database: EV battery metals, + wind power in chemical industry


**4. Electrification of heat in chemical industry**

In [64]:
EI_CHEMICALS_HEAT = "ecoinvent 3.9.1 cutoff, + electrification of heat in chemical industry"

In [37]:
# Change electricity source for heat pump datasets
for ds in ei_db_modif:
    if ds["name"] == heat_filter["name"]:
        # Get wind power dataset for location
        wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, ds["location"], ei_db_modif)

        # Change heat pump electricity source to wind
        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in ['electricity, low voltage',
                                "electricity, medium voltage",
                                "electricity, high voltage"]:
                exc.update({
                    'name': wind_ds_loc['name'],
                    'product': wind_ds_loc['reference product'],
                    'location': wind_ds_loc['location'],
                    'input': (wind_ds_loc['database'], wind_ds_loc['code'])}
            )

In [38]:
# Change heat source in chemical datasets
all_heat_sources = list(set([ds["reference product"] for ds in ei_db_modif if "heat" in ds["reference product"] and ds["unit"] == "megajoule"]))

for ds in ei_db_modif:
    product_category = [i[1] for i in ds["classifications"] if i[0] == 'ISIC rev.4 ecoinvent']

    if len(product_category) > 0:
        if product_category[0] in chemicals_ISIC_categories:

            # Get heat pump dataset
            heat_pump_loc = supporting_functions.get_dataset_for_location(heat_filter, ds["location"], ei_db_modif)
            
            for exc in filter(technosphere, ds["exchanges"]):
                if exc['product'] in all_heat_sources:
                    exc.update({
                        'name': heat_pump_loc['name'],
                        'product': heat_pump_loc['reference product'],
                        'location': heat_pump_loc['location'],
                        'input': (heat_pump_loc['database'], heat_pump_loc['code'])})

In [40]:
# Write the modified ecoinvent database to memory
if EI_CHEMICALS_HEAT in bw.databases:
    del bw.databases[EI_CHEMICALS_HEAT]
wurst.write_brightway2_database(ei_db_modif, EI_CHEMICALS_HEAT)

Vacuuming database 
21238 datasets
674593 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:26


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 12:22:49
  Finished: 12/22/2023 12:24:16
  Total time elapsed: 00:01:26
  CPU %: 93.70
  Memory %: 26.51
Created database: ecoinvent 3.9.1 cutoff, + electrification of heat in chemical industry


In [65]:
# Link metal inventories to the modified ecoinvent version
for ds in metals_db_modif:
    for exc in filter(technosphere, ds["exchanges"]):
        if "ecoinvent" in exc["database"]:
            exc_ds = [
                d for d in ei_db_modif if 
                d["name"] == exc["name"] and
                d["reference product"] == exc["product"] and
                d["location"] == exc["location"] and
                d["unit"] == exc["unit"]
                ][0]
                
            exc.update(
                {
                    "input": (exc_ds["database"], exc_ds["code"])
                }
            )

In [66]:
if dbs_strategies["CHEMICALS_HEAT"] in bw.databases:
    del bw.databases[dbs_strategies["CHEMICALS_HEAT"]]
wurst.write_brightway2_database(metals_db_modif, dbs_strategies["CHEMICALS_HEAT"])

Writing activities to SQLite3 database:


29 datasets
614 exchanges
0 unlinked exchanges
  


0% [#############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 12:45:26
  Finished: 12/22/2023 12:45:26
  Total time elapsed: 00:00:00
  CPU %: 75.60
  Memory %: 5.85
Created database: EV battery metals, + electrification heat in chemical industry


**5. Decarbonization of global power sector**

In [68]:
EI_GLOBAL_POWER = "ecoinvent 3.9.1 cutoff, + wind power in all sectors"

In [40]:
# Change electricity source for all activities
for ds in ei_db_modif:
    # Get wind power dataset for location
    try:
        wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, ds["location"], ei_db_modif)

        # Change heat pump electricity source to wind
        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in ['electricity, low voltage',
                                    "electricity, medium voltage",
                                    "electricity, high voltage"]:
                exc.update({
                    'name': wind_ds_loc['name'],
                    'product': wind_ds_loc['reference product'],
                    'location': wind_ds_loc['location'],
                    'input': (wind_ds_loc['database'], wind_ds_loc['code'])}
            )
    except KeyError:
        pass

CN-SWG not found in regex
CN-CCG not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NWG not found in regex
CN-SWG not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-ECGC not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NCGC not found in regex
RoE not found in ISO3
CN-ECGC not found in regex
CN-CCG not found in regex
CN-ECGC not found in regex
RoE not found in ISO3
CN-ECGC not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NCGC not found in regex
CN-SWG not found in regex
RoE not found in ISO3
CN-ECGC not found in regex
BR-South-eastern/Mid-western grid not found in regex
BR-South-eastern/Mid-western grid not found in regex
BR-South-eastern/Mid-western grid not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-ECGC not found in regex
RoE not found in ISO3
CN-CCG not found in regex
RoE not found in ISO3
CN-ECGC not found in regex
CN-CCG not found in regex
CN-SWG not foun

In [41]:
# Write the modified ecoinvent database to memory
if EI_GLOBAL_POWER in bw.databases:
    del bw.databases[EI_GLOBAL_POWER]
wurst.write_brightway2_database(ei_db_modif, EI_GLOBAL_POWER)

21238 datasets
674593 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:18


Title: Writing activities to SQLite3 database:
  Started: 12/21/2023 17:43:45
  Finished: 12/21/2023 17:45:03
  Total time elapsed: 00:01:18
  CPU %: 88.90
  Memory %: 25.96
Created database: ecoinvent 3.9.1 cutoff, + wind power in all sectors


In [69]:
# Relink metals inventories to the modified ecoinvent database
for ds in metals_db_modif:
    for exc in filter(technosphere, ds["exchanges"]):
        if "ecoinvent" in exc["database"]:
            exc_ds = [
                d for d in bw.Database(EI_GLOBAL_POWER) if 
                d["name"] == exc["name"] and
                d["reference product"] == exc["product"] and
                d["location"] == exc["location"] and
                d["unit"] == exc["unit"]
                ][0]

            exc.update({
                        'input': (exc_ds['database'], exc_ds['code'])}
                        )

In [70]:
if dbs_strategies["GLOBAL_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["GLOBAL_ELEC"]]
wurst.write_brightway2_database(metals_db_modif, dbs_strategies["GLOBAL_ELEC"])

Writing activities to SQLite3 database:


29 datasets
614 exchanges
0 unlinked exchanges
  


0% [#############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/22/2023 12:51:01
  Finished: 12/22/2023 12:51:01
  Total time elapsed: 00:00:00
  CPU %: 80.10
  Memory %: 20.92
Created database: EV battery metals, + wind power in all activities


### Calculte mitigation potential 

In [87]:
mitigation_potential = {
        db: {
            metal: 0 
            for metal in INVENTORIES.keys()} 
            for db in list(dbs_strategies.keys())
        }

for db in dbs_strategies:
    for metal in INVENTORIES:
        metal_ds = [ds for ds in bw.Database(dbs_strategies[db])
                    if ds['name'] == INVENTORIES[metal][0] 
                    and ds['reference product'] == INVENTORIES[metal][1]
                    and ds['location'] == INVENTORIES[metal][2]][0]
        impacts = supporting_functions.multi_lcia(metal_ds, IMPACT_METHODS)
        mitigation_potential[db][metal] = impacts

In [88]:
mitigation_potential = {outer_key: {
    inner_key: sub_dict["Climate change"] 
    for inner_key, sub_dict in inner_dict.items()} 
    for outer_key, inner_dict in mitigation_potential.items()
}

In [89]:
# Export mitigation potential results
pd.DataFrame(mitigation_potential).to_csv(DATA_DIR / "results" / f"fig4_actual_mitigaiton_potential_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Metal")